In [58]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


In [59]:
df = pd.read_csv("players_22.csv")


<ipython-input-59-83330a5928c5>:1: DtypeWarning: Columns (25,108) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("players_22.csv")


In [60]:

df[['attack_wr', 'defense_wr']] = df['work_rate'].str.split('/', expand=True)
wr_map = {'Low': 1, 'Medium': 2, 'High': 3}
df['attack_wr'] = df['attack_wr'].map(wr_map)
df['defense_wr'] = df['defense_wr'].map(wr_map)


df['work_rate_score'] = df[['attack_wr', 'defense_wr']].mean(axis=1)


In [61]:
df.isnull().sum().sort_values(ascending=False)


,0
nation_team_id,18480
nation_position,18480
nation_jersey_number,18480
nation_logo_url,18480
club_loaned_from,18137
...,...
player_face_url,0
nation_flag_url,0
attack_wr,0
defense_wr,0


In [62]:
# Kaleci (GK)
gk_features = [
    'goalkeeping_reflexes',
    'goalkeeping_handling',
    'goalkeeping_diving',
    'goalkeeping_kicking',
    'goalkeeping_positioning',
    'movement_reactions',
    'mentality_composure',
    'height_cm',
    'movement_agility'
]



# Defans (LB/LCB/CB/RCB/RB)
def_features = [
    'defending_standing_tackle',
    'defending_marking_awareness',
    'power_strength',
    'pace',
    'attacking_heading_accuracy'
]

wb_features = [
    'pace',
    'power_stamina',
    'attacking_crossing',
    'defending_standing_tackle',
    'work_rate_score'
]

mid_features = [
    'shooting',
    'passing',
    'dribbling',
    'mentality_vision',
    'work_rate_score'
]

# Forvet (LW/RW/LF/CF/ST, vb.)
st_features = [
    'attacking_finishing',
    'power_strength',
    'mentality_positioning',
    'attacking_heading_accuracy',
    'shooting'
]


In [63]:
def pozisyon_filtresi(df, hedef_kodlari):
    return df[df['player_positions'].str.split(',').str[0].isin(hedef_kodlari)]

pozisyon_kodlari = {
    "GK": (['GK'], gk_features),
    "DEF": (['LB', 'LCB', 'CB', 'RCB', 'RB'], def_features),
    "WB": (['LWB', 'RWB'], wb_features),
    "MID": (['CDM', 'CM', 'CAM', 'LM', 'RM', 'LAM', 'RAM', 'LCM', 'RCM', 'LDM', 'RDM'], mid_features),
    "ST": (['ST', 'CF', 'LF', 'RF', 'LS', 'RS', 'LW', 'RW'], st_features),
}

In [64]:

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, cross_val_score

df = pd.read_csv("players_22.csv")

df[['attack_wr', 'defense_wr']] = df['work_rate'].str.split('/', expand=True)
wr_map = {'Low': 1, 'Medium': 2, 'High': 3}
df['attack_wr'] = df['attack_wr'].map(wr_map)
df['defense_wr'] = df['defense_wr'].map(wr_map)
df['work_rate_score'] = df[['attack_wr', 'defense_wr']].mean(axis=1)

gk_features = [
    'goalkeeping_reflexes',
    'goalkeeping_handling',
    'goalkeeping_diving',
    'goalkeeping_kicking',
    'goalkeeping_positioning',
    'movement_reactions',
    'mentality_composure',
    'height_cm',
    'movement_agility'
]


def_features = [
    'defending_standing_tackle', 'defending_marking_awareness',
    'power_strength', 'pace', 'attacking_heading_accuracy'
]

wb_features = [
    'pace', 'power_stamina', 'attacking_crossing',
    'defending_standing_tackle', 'work_rate_score'
]

mid_features = [
    'shooting', 'passing', 'dribbling', 'mentality_vision', 'work_rate_score'
]

st_features = [
    'attacking_finishing', 'power_strength', 'mentality_positioning',
    'attacking_heading_accuracy', 'shooting'
]


# Pozisyona Göre Filtreleme
def pozisyon_filtresi(df, hedef_kodlari):
    return df[df['player_positions'].str.split(',').str[0].isin(hedef_kodlari)]

pozisyon_kodlari = {
    "GK": (['GK'], gk_features),
    "DEF": (['LB', 'LCB', 'CB', 'RCB', 'RB'], def_features),
    "WB": (['LWB', 'RWB'], wb_features),
    "MID": (['CDM', 'CM', 'CAM', 'LM', 'RM', 'LAM', 'RAM', 'LCM', 'RCM', 'LDM', 'RDM'], mid_features),
    "ST": (['ST', 'CF', 'LF', 'RF', 'LS', 'RS', 'LW', 'RW'], st_features),
}


#Model Eğitimi + Çapraz Doğrulama

cv = KFold(n_splits=5, shuffle=True, random_state=42)

for poz_etiket, (poz_kodlar, ozellikler) in pozisyon_kodlari.items():
    subset = pozisyon_filtresi(df, poz_kodlar)
    X = subset[ozellikler].dropna()
    y = subset.loc[X.index, 'overall']

    if len(X) < 100:
        print(f"{poz_etiket} için yeterli veri yok ({len(X)} satır), atlanıyor.\n")
        continue

    model = RandomForestRegressor(n_estimators=100, random_state=42)

    r2 = cross_val_score(model, X, y, cv=cv, scoring='r2')
    mae = -cross_val_score(model, X, y, cv=cv, scoring='neg_mean_absolute_error')
    rmse = np.sqrt(-cross_val_score(model, X, y, cv=cv, scoring='neg_mean_squared_error'))

    print(f" {poz_etiket} Modeli - 5 Katlı Çapraz Doğrulama")
    print(f"R² Ort.:    {r2.mean():.4f}")
    print(f"MAE Ort.:   {mae.mean():.4f}")
    print(f"RMSE Ort.:  {rmse.mean():.4f}")
    print("-" * 45)


<ipython-input-64-06520dd7d8de>:12: DtypeWarning: Columns (25,108) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("players_22.csv")


📌 GK Modeli - 5 Katlı Çapraz Doğrulama
R² Ort.:    0.9933
MAE Ort.:   0.4526
RMSE Ort.:  0.6168
---------------------------------------------
📌 DEF Modeli - 5 Katlı Çapraz Doğrulama
R² Ort.:    0.9179
MAE Ort.:   1.4358
RMSE Ort.:  1.8736
---------------------------------------------
📌 WB Modeli - 5 Katlı Çapraz Doğrulama
R² Ort.:    0.9310
MAE Ort.:   1.2539
RMSE Ort.:  1.6476
---------------------------------------------
📌 MID Modeli - 5 Katlı Çapraz Doğrulama
R² Ort.:    0.8339
MAE Ort.:   2.0684
RMSE Ort.:  2.7929
---------------------------------------------
📌 ST Modeli - 5 Katlı Çapraz Doğrulama
R² Ort.:    0.8874
MAE Ort.:   1.7462
RMSE Ort.:  2.3391
---------------------------------------------


In [65]:
# Pozisyon bazlı özellikler ve modeller
pozisyon_ozellikleri = {
    "GK": {
        "features": [
            ('goalkeeping_reflexes', 'Refleks - Yakın mesafe şutlara hızlı tepki'),
            ('goalkeeping_handling', 'Top Tutma - Topu sektirmeme becerisi'),
            ('goalkeeping_diving', 'Uzanma - Yana atlama ve kurtarış kabiliyeti'),
            ('goalkeeping_kicking', 'Şut/Pas - Ayakla topu iyi oynama'),
            ('goalkeeping_positioning', 'Pozisyon Alma - Doğru yerde durma'),
            ('movement_reactions', 'Tepki - Ani durumlara tepki süresi'),
            ('mentality_composure', 'Soğukkanlılık - Baskı altında karar kalitesi'),
            ('height_cm', 'Boy (cm)'),
            ('movement_agility', 'Çeviklik - Hızlı yön değiştirme')
        ],
        "model": gk_model
    },
    "DEF": {
        "features": [
            ('defending_standing_tackle', 'Müdahale - Ayakta top alma becerisi'),
            ('defending_marking_awareness', 'Markaj - Adam takibi ve yerleşim'),
            ('power_strength', 'Güç - Fiziksel mücadele seviyesi'),
            ('pace', 'Hız - Topsuz sürat'),
            ('attacking_heading_accuracy', 'Kafa - Hava topu hakimiyeti')
        ],
        "model": def_model
    },
    "WB": {
        "features": [
            ('pace', 'Hız - Topsuz sürat'),
            ('power_stamina', 'Dayanıklılık - Maç boyu koşabilme'),
            ('attacking_crossing', 'Orta Açma - Kenardan etkili pas'),
            ('defending_standing_tackle', 'Müdahale - Savunma becerisi'),
            ('work_rate_score', 'Çalışkanlık - Ortalama iş gücü skoru (1=Low, 2=Medium, 3=High)')
        ],
        "model": wb_model
    },
    "MID": {
        "features": [
            ('shooting', 'Şut - Genel şut becerisi'),
            ('passing', 'Pas - Oyun kurma kabiliyeti'),
            ('dribbling', 'Top Sürme - Rakip eksiltme'),
            ('mentality_vision', 'Oyun Görüşü - Pas kanallarını görme'),
            ('work_rate_score', 'Çalışkanlık - Ortalama iş gücü skoru (1=Low, 2=Medium, 3=High)')
        ],
        "model": mid_model
    },
    "ST": {
        "features": [
            ('attacking_finishing', 'Bitiricilik - Gol vuruş kalitesi'),
            ('power_strength', 'Güç - Fiziksel dayanıklılık'),
            ('mentality_positioning', 'Pozisyon Alma - Boş alan bulma'),
            ('attacking_heading_accuracy', 'Kafa - Hava topu etkisi'),
            ('shooting', 'Şut - Genel gol tehditi')
        ],
        "model": st_model
    }
}

#  Kullanıcıdan giriş alıp tahmin yapan fonksiyon
def pozisyona_gore_tahmin():
    secenekler = list(pozisyon_ozellikleri.keys())
    pozisyon = input(f"Pozisyon seç ({'/'.join(secenekler)}): ").strip().upper()

    if pozisyon not in pozisyon_ozellikleri:
        print(" Geçersiz pozisyon! Lütfen şunlardan birini yaz: GK, DEF, WB, MID, ST")
        return

    ozellik_listesi = pozisyon_ozellikleri[pozisyon]['features']
    model = pozisyon_ozellikleri[pozisyon]['model']

    print(f"\n Seçilen pozisyon: {pozisyon}")
    print("Lütfen aşağıdaki özellik puanlarını girin (0-100 arası):\n")

    veri = {}
    for kod, aciklama in ozellik_listesi:
        while True:
            try:
                val = float(input(f"{aciklama}: "))
                if 'height' in kod:
                    veri[kod] = val
                    break
                elif 0 <= val <= 100:
                    veri[kod] = val
                    break
                else:
                    print("0 ile 100 arasında bir değer gir.")
            except:
                print("Geçerli bir sayı gir.")

    df_girdi = pd.DataFrame([veri])
    tahmin = model.predict(df_girdi)[0]
    print(f"\n Tahmini Overall: {round(tahmin, 2)}")

pozisyona_gore_tahmin()


Pozisyon seç (GK/DEF/WB/MID/ST): MID

📋 Seçilen pozisyon: MID
Lütfen aşağıdaki özellik puanlarını girin (0-100 arası):

Şut - Genel şut becerisi: 82
Pas - Oyun kurma kabiliyeti: 79
Top Sürme - Rakip eksiltme: 88
Oyun Görüşü - Pas kanallarını görme: 89
Çalışkanlık - Ortalama iş gücü skoru (1=Low, 2=Medium, 3=High): 2

🎯 Tahmini Overall: 85.91


In [67]:
import pickle
with open("gk_model.pkl", "wb") as f:
    pickle.dump(gk_model, f)

with open("def_model.pkl", "wb") as f:
    pickle.dump(def_model, f)

with open("wb_model.pkl", "wb") as f:
    pickle.dump(wb_model, f)

with open("mid_model.pkl", "wb") as f:
    pickle.dump(mid_model, f)

with open("st_model.pkl", "wb") as f:
    pickle.dump(st_model, f)
import os
import joblib
os.makedirs("models", exist_ok=True)
joblib.dump(gk_model, "models/gk_model.pkl")
joblib.dump(def_model, "models/def_model.pkl")
joblib.dump(wb_model, "models/wb_model.pkl")
joblib.dump(mid_model, "models/mid_model.pkl")
joblib.dump(st_model, "models/st_model.pkl")
from google.colab import files
files.download("models/gk_model.pkl")
files.download("models/def_model.pkl")
files.download("models/wb_model.pkl")
files.download("models/mid_model.pkl")
files.download("models/st_model.pkl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>